In [2]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
# from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Dropout, Flatten,BatchNormalization
from keras.layers import Input, Conv2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
from keras.callbacks import EarlyStopping

# import warnings
# warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from PIL import Image
import os
from scipy import misc
from  collections import Counter


In [3]:
PATH_TO_IMAGES="/home/roman/DS_part/image_50_array/"

### Работа с данными

In [4]:
#Загрузка данных
X=np.load(PATH_TO_IMAGES+"X_data.npy")
y=np.load(PATH_TO_IMAGES+"y_data.npy")

In [5]:
#Кодирую имена числами
la=LabelEncoder()
y_copy=y.copy()
y=la.fit_transform(y)

In [6]:
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=17,stratify=y)

In [7]:
num_train, height, width,depth = X_train.shape 
num_test = X_test.shape[0]
num_classes = np.unique(y_train).shape[0] 

#new
X_train =X_train/255 # Normalise data to [0, 1] range
X_test =X_test/255 # Normalise data to [0, 1] range


Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [10]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 30 # we iterate 30 times over the entire training set
kernel_size = (3,3) # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

### VGG architecture

In [14]:
inp = Input(shape=(128, 128,3))

pad1_1 = ZeroPadding2D(padding=(1, 1))(inp)
conv1_1 = Conv2D(64, 3, 3, activation='relu', name='conv1_1')(pad1_1)
pad1_2 = ZeroPadding2D(padding=(1, 1))(conv1_1)
conv1_2 = Conv2D(64, 3, 3, activation='relu', name='conv1_2')(pad1_2)
pool1 = MaxPooling2D((2, 2), strides=(2, 2))(conv1_2)

pad2_1 = ZeroPadding2D((1, 1))(pool1)
conv2_1 = Conv2D(128, 3, 3, activation='relu', name='conv2_1')(pad2_1)
pad2_2 = ZeroPadding2D((1, 1))(conv2_1)
conv2_2 = Conv2D(128, 3, 3, activation='relu', name='conv2_2')(pad2_2)
pool2 = MaxPooling2D((2, 2), strides=(2, 2))(conv2_2)

pad3_1 = ZeroPadding2D((1, 1))(pool2)
conv3_1 = Conv2D(256, 3, 3, activation='relu', name='conv3_1')(pad3_1)
pad3_2 = ZeroPadding2D((1, 1))(conv3_1)
conv3_2 = Conv2D(256, 3, 3, activation='relu', name='conv3_2')(pad3_2)
pad3_3 = ZeroPadding2D((1, 1))(conv3_2)
conv3_3 = Conv2D(256, 3, 3, activation='relu', name='conv3_3')(pad3_3)
pool3 = MaxPooling2D((2, 2), strides=(2, 2))(conv3_3)

pad4_1 = ZeroPadding2D((1, 1))(pool3)
conv4_1 = Conv2D(512, 3, 3, activation='relu', name='conv4_1')(pad4_1)
pad4_2 = ZeroPadding2D((1, 1))(conv4_1)
conv4_2 = Conv2D(512, 3, 3, activation='relu', name='conv4_2')(pad4_2)
pad4_3 = ZeroPadding2D((1, 1))(conv4_2)
conv4_3 = Conv2D(512, 3, 3, activation='relu', name='conv4_3')(pad4_3)
pool4 = MaxPooling2D((2, 2), strides=(2, 2))(conv4_3)

pad5_1 = ZeroPadding2D((1, 1))(pool4)
conv5_1 = Conv2D(512, 3, 3, activation='relu', name='conv5_1')(pad5_1)
pad5_2 = ZeroPadding2D((1, 1))(conv5_1)
conv5_2 = Conv2D(512, 3, 3, activation='relu', name='conv5_2')(pad5_2)
pad5_3 = ZeroPadding2D((1, 1))(conv5_2)
conv5_3 = Conv2D(512, 3, 3, activation='relu', name='conv5_3')(pad5_3)
pool5 = MaxPooling2D((2, 2), strides=(2, 2))(conv5_3)

flat = Flatten()(pool5)
fc6 = Dense(4096, activation='relu', name='fc6')(flat)
fc6_drop = Dropout(0.5)(fc6)
fc7 = Dense(4096, activation='relu', name='fc7')(fc6_drop)
fc7_drop = Dropout(0.5)(fc7)
out = Dense(num_classes, activation='softmax', name='fc8')(fc7_drop)

model = Model(inputs=inp, outputs=out)



model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy


earlyStopping=EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')


model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=2, validation_split=0.1,callbacks=[earlyStopping]) # ...holding out 10% of the data for validation


model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

/home/roman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_1")`
  after removing the cwd from sys.path.
/home/roman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu", name="conv1_2")`
  
/home/roman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_1")`
  # Remove the CWD from sys.path while we load stuff.
/home/roman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu", name="conv2_2")`
  if sys.path[0] == '':
/home/roman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv2D` call 

Train on 49197 samples, validate on 5467 samples
Epoch 1/30
 - 1223s - loss: 7.8693 - acc: 1.6261e-04 - val_loss: 7.8700 - val_acc: 1.8292e-04
Epoch 2/30
 - 1211s - loss: 7.8599 - acc: 4.4718e-04 - val_loss: 7.8718 - val_acc: 3.6583e-04
23428/23428 [==============================] - 160s 7ms/step


[7.855263242710159, 0.000597575550623186]

In [ ]:
#Вывод: либо я не VGG импользую либо херня какая то а не архитектура